In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import requests
from io import StringIO

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get('https://www.giallozafferano.it/ricette-cat/')

    # Attendimo l'eventuale popup dei cookie
    try:
        cookie_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='Continua senza accettare']"))
        )
        cookie_button.click()
    except:
        print("Nessun popup dei cookie trovato.")

    tutte_ricette = []

    while True:
        # Scorre la pagina verso il basso gradualmente per assicurare che tutti gli articoli siano visibili
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Trova tutti gli articoli delle ricette
        articoli = driver.find_elements(By.CLASS_NAME, 'gz-card')

        # Itera attraverso ogni articolo per estrarre nome, categoria e ingredienti
        for i in range(len(articoli)):
            # Ricarica la pagina principale e trova di nuovo tutti gli articoli
            articoli = driver.find_elements(By.CLASS_NAME, 'gz-card')

            try:
                # Estrai il titolo e la categoria dell'articolo corrente
                titolo = articoli[i].find_element(By.CLASS_NAME, 'gz-title').text
                categoria = articoli[i].find_element(By.CLASS_NAME, 'gz-category').text

                # Trova il link della ricetta usando XPath e naviga a quella pagina
                link_ricetta = articoli[i].find_element(By.XPATH, ".//a[@title]").get_attribute('href')
                if link_ricetta is not None:
                    driver.get(link_ricetta)

                # Attendi che la pagina degli ingredienti si carichi
                wait = WebDriverWait(driver, 2)
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'gz-ingredients')))

                # Trova tutti gli elementi degli ingredienti
                ingredienti_section = driver.find_element(By.CLASS_NAME, 'gz-ingredients')
                ingredienti_list = ingredienti_section.find_elements(By.CLASS_NAME, 'gz-ingredient')

                # Estrai il testo degli ingredienti
                ingredienti_lista = [ingrediente.text for ingrediente in ingredienti_list]

                # Salva il titolo, la categoria e gli ingredienti
                tutte_ricette.append({'titolo': titolo, 'categoria': categoria, 'ingredienti': ingredienti_lista})
                driver.back()
                time.sleep(3)
            except Exception as e:
                print(f"Errore nell'elaborazione di un articolo: {e}")

        # Trova il pulsante "Pagina successiva" e cliccalo se esiste, altrimenti esci dal ciclo
        try:
            next_button = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'gz-arrow.next'))
            )
            next_button.click()
            time.sleep(2)
        except:
            print("Non ci sono più pagine da processare.")
            break
    for ricetta in tutte_ricette:
        ricetta['ingredienti'] = [ingrediente.replace("Olio extravergine d'oliva", "Olio extravergine di oliva") for ingrediente in ricetta['ingredienti']]

except Exception as e:
    print(f"Si è verificato un errore: {e}")
finally:
    driver.quit()

ricette = pd.DataFrame(tutte_ricette)

# Prove

In [16]:
# Define the URL of the CSV file
url = 'https://raw.githubusercontent.com/andrecardinali/dataman/main/Ricette.csv'

# Get the content of the URL
response = requests.get(url)

# Decode the content to a string
data = StringIO(response.text)

# Read the CSV data from the string
ricette = pd.read_csv(data)

In [17]:
def parse_ingredient(ingredient):
    parts = ingredient.split()
    if len(parts) > 2 and (parts[-2].isdigit() and parts[-1] in ['g', 'ml', 'l', 'kg', 'cl', 'gr']):
        quantity = ' '.join(parts[-2:])
        name = ' '.join(parts[:-2])
    else:
        quantity = parts[-1]
        name = ' '.join(parts[:-1])
    return name, quantity

def convert_ingredient_list_to_dict(ingredient_list):
    ingredient_dict = {}
    for ingredient in ingredient_list:
        name, quantity = parse_ingredient(ingredient)
        ingredient_dict[name] = quantity
    return ingredient_dict

# Rimuovere le parentesi quadre e i singoli apici dalle stringhe, assicurandosi che i valori siano stringhe

ricette['ingredienti'].str.replace(" ' ", "i ", regex=False)
ricette['ingredienti'] = ricette['ingredienti'].astype(str).str.strip("[]").str.replace("'", "")

# Convertire la stringa di ingredienti in una lista
ricette['ingredienti'] = ricette['ingredienti'].apply(lambda x: x.split(", "))

# Convertire la lista di ingredienti in un dizionario
ricette['ingredienti_dict'] = ricette['ingredienti'].apply(convert_ingredient_list_to_dict)

# Visualizzare le prime righe del DataFrame aggiornato
#ricette.drop('ingredienti', axis=1, inplace=True)
ricette.head()

,titolo,categoria,ingredienti,ingredienti_dict
0,Budino al gianduia con salsa all'arancia,DOLCI,"[Panna fresca liquida 150 g, Amido di mais (ma...","{'Panna fresca liquida': '150 g', 'Amido di ma..."
1,Sfogliatine di pere con Nutella®,DOLCI,"[Nutella® 75 g, Pere Williams 1, Pasta Sfoglia...","{'Nutella®': '75 g', 'Pere Williams': '1', 'Pa..."
2,Monodose natalizia,DOLCI,"[Burro a temperatura ambiente 100 g, Zucchero ...","{'Burro a temperatura ambiente': '100 g', 'Zuc..."
3,Polpettine mostruose,SECONDI PIATTI,"[Ricotta di vaccina 175 gr, Parmigiano Reggian...","{'Ricotta di vaccina': '175 gr', 'Parmigiano R..."
4,Friselle con cotechino e lenticchie,ANTIPASTI,"[Friselle da 30 grammi ciascuna 4, Cotechino p...","{'Friselle da 30 grammi ciascuna': '4', 'Cotec..."


In [ ]:
ricette.drop('ingredienti', axis=1, inplace=True)

In [ ]:
ricette.to_csv('Ricette.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
ricette.to_json('Ricette.json', orient='records', lines=True)